In [19]:
import numpy as np

In [20]:
def generate_time_series(batch_size, n_steps):
    freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
    time = np.linspace(0, 1, n_steps)
    series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10))  #   웨이브 1
    series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + 웨이브 2
    series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5)   # + 잡음
    return series[..., np.newaxis].astype(np.float32)

batch_size 개수만큼의 시계열 데이터가 생성

각 시계열 데이터는 n_steps 만큼의 데이터가 있고,

앞선 n_steps - 1 개의 데이터를 통해 n_steps 번째의 데이터를 예측한다.

In [21]:
np.random.seed(42)

n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [22]:
X_train.shape

(7000, 50, 1)

In [23]:
X_train[0].shape

(50, 1)

In [24]:
y_train.shape

(7000, 1)

In [25]:
# 활성화 함수
class activation_function:
  # 시그모이드 함수
  def sigmoid(self, x):
    return 1 / (1+np.e**-x)
  
  # 시그모이드 함수의 미분 함수
  def sigmoid_diff(self, x):
    return x * (1 - x)

In [26]:
# 비용 함수 클래스, 비용 함수의 종류에 따라 다른 클래스 내 함수를 사용한다.
class cost_function:
  # 예측값
  predict = []
  # 타겟값
  target = []
  # 비용 함수값
  error_cost = []

  # 오차 제곱합
  def errer_squared_sum(self, predict, target):
    self.predict = predict
    self.target = target

    self.error_cost = np.sum(0.5*((predict - target)**2))
    return self.error_cost

  # 오차 제곱합 미분 함수
  def diff_error_squared_sum(self):
    return (self.predict[0][0] - self.target[0][0])

In [27]:
class RNN_vector_to_sequence:
    #입력값
    input_data = []
    
    #가중치
    weight = []
    
    #편향값
    bias = []

    # 입력 가중치
    input_w = []

    # 이전 출력에 대한 가중치
    before_w = []

    # 편향 값 bais
    bias = []

    # 노드 입력
    node_input = []

    # 노드 출력
    node_output = []

    # 타겟, 목푯값
    target = []

    # 활성화 함수
    activation = activation_function()

    # 비용 함수
    cost = cost_function()

    # 예측 결과
    predict = []

    # 각 층의 델타 값의 저장
    delta = []

     # 각 가중치 업데이트 크기
    input_weight_update = []

    before_weight_update = []

    bias_update = []

    def delta_to_sequence_cal_rnn_test(self, input, target):

      # 타겟값의 저장
      self.target = target

      # 입력 데이터의 저장
      self.input = input

      # 입력 데이터 크기에 맞는 가중치 값이 생성되어야 한다.
      # 입력 (3,1) 데이터에 대한 연산 결과 (1,1)를 예측한다고 가정,

      # 입력값에 사용되는 가중치
      # (3,1) 크기의 입력 데이터에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위해, (1,3) 크기의 가중치 행렬이 필요하다
      self.input_w = np.random.rand(target.shape[0], input.shape[1])
      
      # 이전 출력에 대한 가중치
      # (1,1) 노드 출력에 대해 (1,1) 의 행렬곱 연산 결과를 얻기 위한, (1,1) 크기의 가중치 행렬
      self.before_w = np.random.rand(target.shape[0], target.shape[0])

      # bias 값 생성, (1,1) 행렬에 더해지기 위한 (1,1) 크기의 bias 값 생성
      self.bias = np.random.rand(target.shape[0], target.shape[0])

      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((target.shape[0], target.shape[0]))

      self.node_output.append(before)
      
      # (3, n) 크기의 입력 데이터에서 n 번의 반복을 수행하게 됨
      for i in range(input.shape[0]):
        # 노드 입력 데이터의 numpy 행렬 생성
        input_data = input[i].reshape(input.shape[1],-1)

        # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
        node_input = (self.before_w @ before) + (self.input_w @ input_data) + self.bias
        self.node_input.append(node_input)

        # 노드 출력 계산, 활성화 함수 연산을 수행한다.
        node_output = self.activation.sigmoid(node_input)
        self.node_output.append(node_output)

        # 타입 스텝의 출력이 다음 노드의 입력이 된다.
        before = node_output

      self.predict = node_output

      self.cost_cal()

      return self.predict

    def cost_cal(self):
      return self.cost.errer_squared_sum(self.predict, self.target)

    
    def cal_delta(self):
      #마지막 노드의 변화량에 대한 오차 함수의 변화량 계산
      delta = (self.cost.diff_error_squared_sum() * self.activation.sigmoid_diff(self.predict))
      self.delta.append(delta)

      # n개의 delta 값을 계산하기 위한 반복
      for i in range(input.shape[0] - 1, -1, -1):
        # 노드 출력의 변화에 대한 비용 함수의 변화량 계산
        delta = self.before_w @ delta
        # 활성화 함수에 따른 변화량 계산, 해당 값이 노드 변화에 대한 비용 함수의 변화량이 된다. 
        delta = delta * self.activation.sigmoid_diff(self.node_output[self.input.shape[0] - 1])
        # 노드별 delta 값의 저장
        self.delta.append(delta)
    

    def update_weight(self, learning_rate):
      # 원활한 연산을 위해 마지막 노드부터 거꾸로 계산된 delta 값을 뒤집어준다.
      self.delta = self.delta[::-1]

      result = 0

      # 해당 delta 값과의 데이터 입력값을 통해 input_w 의 가중치 변화량을 계산할 수 있다
      for i in range(self.input.shape[0]):
        result = result + self.delta[i] * self.node_input[i]

      self.before_weight_update = result

      # 이전 노드 출력값을 통해 before_w 의 가중치 변화량 계산
      result = 0

      for i in range(self.input.shape[0]):
        result = result + self.delta[i] * self.input[i]

      self.input_weight_update = result

      # bias_update 의 계산
      result = 0
      
      for i in range(self.input.shape[0]):
        result = result + self.delta[i]

      self.bias_update = result

      # 가중치 업데이트
      self.input_w = self.input_w - (self.input_weight_update * learning_rate)

      self.before_w = self.before_w - (self.before_weight_update * learning_rate)

      self.bias = self.bias - self.bias_update * learning_rate

    def iterations(self, iterations, learning_rate):
      # 이전 노드의 출력, 첫 부분의 경우 이 값이 0이다. 행렬의 크기는 (1,1)
      before = np.zeros((self.target.shape[0], self.target.shape[0]))
      for j in range(iterations):
        # 초기화
        self.node_input = []
        self.node_output = []
        for i in range(input.shape[0]):
          # 노드 입력 데이터의 numpy 행렬 생성
          input_data = input[i].reshape(input.shape[1],-1)

          # RNN 노드 입력 값 계산, 이전 노드 출력 @ 가중치 + 입력 데이터 @ 가중치
          node_input = (self.before_w @ before) + (self.input_w @ input_data) + self.bias
          self.node_input.append(node_input)

          # 노드 출력 계산, 활성화 함수 연산을 수행한다.
          node_output = self.activation.sigmoid(node_input)
          self.node_output.append(node_output)

          # 순환 노드의 다음 노드 입력값의 계산
          before = node_output

        self.predict = before
        self.cal_delta()
        self.update_weight(learning_rate)
        print(self.cost_cal(), self.predict, self.target)



In [38]:
rnn = RNN_vector_to_sequence()

In [39]:
input = X_train[0]

In [40]:
target = y_train[0].reshape(1,-1)

In [43]:
rnn.delta_to_sequence_cal_rnn_test(input, target)

array([[0.55821308]])

In [44]:
rnn.cal_delta()

In [45]:
rnn.bias

array([[0.16045254]])

In [46]:
rnn.iterations(2, 0.001)

0.41688225578539584 [[0.55821308]] [[-0.35489398]]
0.41687811429954796 [[0.55820855]] [[-0.35489398]]


In [47]:
rnn.iterations(10, 0.01)

0.4168739729161452 [[0.55820401]] [[-0.35489398]]
0.41683256113385325 [[0.55815866]] [[-0.35489398]]
0.4167911598004308 [[0.55811331]] [[-0.35489398]]
0.41674977075993275 [[0.55806798]] [[-0.35489398]]
0.41670839400434384 [[0.55802266]] [[-0.35489398]]
0.41666702952556217 [[0.55797734]] [[-0.35489398]]
0.41662567731549155 [[0.55793204]] [[-0.35489398]]
0.4165843373660415 [[0.55788676]] [[-0.35489398]]
0.4165430096691273 [[0.55784148]] [[-0.35489398]]
0.41650169421666966 [[0.55779621]] [[-0.35489398]]
